# Install and Import Dependencies

In [1]:
%pip install mediapipe opencv-python
%pip install ultralytics supervision


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for supervision from https://files.pythonhosted.org/packages/c0/59/6bc863f530c0ba5da4c565fd40f17b2b333c2ab28b783202937293ae494a/supervision-0.16.0-py3-none-any.whl.metadata
  Using cached supervision-0.16.0-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for opencv-python-headless<5.0.0.0,>=4.8.0.74 from https://files.pythonhosted.org/packages/12/0f/b87324db284c54d1d1a1c1242a128fb18515915d124325784c90f23d8ef5/opencv_python_headless-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl.metadata
  Using cached opencv_python_headless-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl.metadata (19 kB)
Using cached supervision-0.16.0-py3-none-any.whl (72 kB)
Using cached opencv_python_headless-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl (33.1 MB)

[notice] A new release of pip is avail

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
from ultralytics import YOLO
import supervision as sv

In [3]:
# import uuid
# import os
# mp_hands = mp.solutions.hands

# Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

<img src="https://i.imgur.com/qpRACer.png" style="height:300px" >

In [ ]:
# for ldmrk in mp_hands.HandLandmark:
#     print(ldmrk)

# Calculate Angles

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# Pose Detection

In [5]:
def display_hand(image, hand_pts):
    # DISPLAY AREA OF RIGHT HAND ---------------
    # Calculate the center of the circle in 3D space (x, y, z)
    center_3d = np.mean(hand_pts, axis=0)
    # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
    distances = [np.linalg.norm([p[0] - center_3d[0], p[1] - center_3d[1], p[2] - center_3d[2]]) for p in hand_pts]

    scaling_factor = 2  # scaling factor must be int
    radius_3d = scaling_factor * int(sum(distances ) / len(distances))

    # Draw the circle in 3D space
    cv2.circle(image, (int(center_3d[0]), int(center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle
    

In [6]:
def feet_pts(ankle, heel, index, frame_shape_1, frame_shape_0):
    return np.array([
        [int(ankle.x * frame_shape_1), int(ankle.y * frame_shape_0)],
        [int(heel.x * frame_shape_1), int(heel.y * frame_shape_0)],
        [int(index.x * frame_shape_1), int(index.y * frame_shape_0)]
        ], np.int32).reshape((-1, 1, 2))

In [7]:
def hand_pts(pinky, index, thumb, wrist, frame_shape_1, frame_shape_0):
    return np.array([
        [int(pinky.x * frame_shape_1), int(pinky.y * frame_shape_0), pinky.z],
        [int(index.x * frame_shape_1), int(index.y * frame_shape_0), index.z],
        [int(thumb.x * frame_shape_1), int(thumb.y * frame_shape_0), thumb.z],
        [int(wrist.x * frame_shape_1), int(wrist.y * frame_shape_0), wrist.z]
        ], np.int32)

In [8]:
def display_coords(d):
    max_key_length = max(len(key) for key in d.keys())
    max_x_length = max(len(str(value.x)) for value in d.values())
    max_y_length = max(len(str(value.y)) for value in d.values())
    max_z_length = max(len(str(value.z)) for value in d.values())

    for point, coords in d.items():
        formatted_x = str(coords.x).rjust(max_x_length)
        formatted_y = str(coords.y).rjust(max_y_length)
        formatted_z = str(coords.z).rjust(max_z_length)
        print(f"{point.ljust(max_key_length)}: x = {formatted_x}, y = {formatted_y}, z = {formatted_z}")
    print("\n")

In [9]:
# JUST THE POSE
cap = cv2.VideoCapture(1)

model = YOLO('bestHuge.pt')
box_annotator = sv.BoxAnnotator(thickness=2, text_thickness=2, text_scale=1)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)


        # YOLO results
        results2 = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results2)
        detections = detections[detections.confidence > 0.8]
        
        frame = box_annotator.annotate(scene=image, detections=detections)
        

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
        #                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            pose_landmark = mp_pose.PoseLandmark

            d = {}  # body dictionary
            
            # Upper body coordinates
            d["left_shoulder"] = landmarks[pose_landmark.LEFT_SHOULDER.value]
            d["right_shoulder"] = landmarks[pose_landmark.RIGHT_SHOULDER]
            
            d["left_elbow"] = landmarks[pose_landmark.LEFT_ELBOW.value]
            d["right_elbow"] = landmarks[pose_landmark.RIGHT_ELBOW.value]

            frame_shape_0, frame_shape_1 = frame.shape[0], frame.shape[1]

            # LEFT HAND
            d["left_pinky"] = landmarks[pose_landmark.LEFT_PINKY.value]
            d["left_index"] = landmarks[pose_landmark.LEFT_INDEX.value]
            d["left_thumb"] = landmarks[pose_landmark.LEFT_THUMB.value]
            d["left_wrist"] = landmarks[pose_landmark.LEFT_WRIST.value]
            left_hand_pts = hand_pts(d["left_pinky"], d["left_index"], d["left_thumb"], d["left_wrist"], frame_shape_1, frame_shape_0)

            # RIGHT HAND
            d["right_pinky"] = landmarks[pose_landmark.RIGHT_PINKY.value]
            d["right_index"] = landmarks[pose_landmark.RIGHT_INDEX.value]
            d["right_thumb"] = landmarks[pose_landmark.RIGHT_THUMB.value]
            d["right_wrist"] = landmarks[pose_landmark.RIGHT_WRIST.value]
            right_hand_pts = hand_pts(d["right_pinky"], d["right_index"], d["right_thumb"], d["right_wrist"], frame_shape_1, frame_shape_0)

            # Lower body coordinates
            d["left_knee"] = landmarks[pose_landmark.LEFT_KNEE.value]
            d["right_knee"] = landmarks[pose_landmark.RIGHT_KNEE.value]

            # LEFT FOOT
            d["left_ankle"] = landmarks[pose_landmark.LEFT_ANKLE.value]
            d["left_heel"] = landmarks[pose_landmark.LEFT_HEEL.value]
            d["left_foot_index"] = landmarks[pose_landmark.LEFT_FOOT_INDEX.value]
            left_foot_pts = feet_pts(d["left_ankle"], d["left_heel"], d["left_foot_index"], frame_shape_1, frame_shape_0)

            # RIGHT FOOT
            d["right_ankle"] = landmarks[pose_landmark.RIGHT_ANKLE.value]
            d["right_heel"] = landmarks[pose_landmark.RIGHT_HEEL.value]
            d["right_foot_index"] = landmarks[pose_landmark.RIGHT_FOOT_INDEX.value]
            right_foot_pts = feet_pts(d["right_ankle"], d["right_heel"], d["right_foot_index"], frame_shape_1, frame_shape_0)

            # Display Coordinates
            display_coords(d)
            
            cv2.fillPoly(image, [right_foot_pts], (245, 117, 66))
            cv2.fillPoly(image, [left_foot_pts], (245, 117, 66))

            display_hand(image,right_hand_pts)
            display_hand(image,left_hand_pts)
        
            # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            # cv2.putText(image, str(angle),
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

        except:
            pass

        # print(results.pose_landmarks)
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
        #                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Pose Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

2023-10-31 15:25:09.272 Python[54490:10358107] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.

0: 384x640 2 0s, 45.9ms
Speed: 2.5ms preprocess, 45.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
2023-10-31 15:25:28.056 Python[54490:10358107] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.

0: 384x640 (no detections), 42.9ms
Speed: 2.1ms preprocess, 42.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 42.4ms
Speed: 2.1ms preprocess, 42.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 46.0ms
Speed: 1.9ms preprocess, 46.0ms inference, 0.3ms postproce

left_shoulder   : x =  0.939335823059082, y = 0.10364390909671783, z =  -0.04198605194687843
right_shoulder  : x = 0.8868051767349243, y = 0.12672953307628632, z = -0.004832190461456776
left_elbow      : x = 0.9649419784545898, y = 0.20178690552711487, z = -0.012386258691549301
right_elbow     : x = 0.8938024640083313, y = 0.23678681254386902, z =  0.016442807391285896
left_pinky      : x = 0.9468657970428467, y =   0.254242867231369, z = -0.009075683541595936
left_index      : x = 0.9436585903167725, y = 0.24086244404315948, z = -0.009839365258812904
left_thumb      : x = 0.9431560635566711, y =  0.2370176911354065, z = -0.005199186969548464
left_wrist      : x = 0.9499510526657104, y = 0.23783095180988312, z = -0.006337944883853197
right_pinky     : x = 0.9118759036064148, y = 0.31704121828079224, z = -0.008116544224321842
right_index     : x = 0.9099021553993225, y = 0.30976712703704834, z =  -0.02201942913234234
right_thumb     : x = 0.9109159111976624, y = 0.30222970247268677, z =

0: 384x640 3 0s, 4 1s, 45.0ms
Speed: 2.0ms preprocess, 45.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 0s, 4 1s, 47.1ms
Speed: 1.9ms preprocess, 47.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 0s, 5 1s, 47.3ms
Speed: 2.0ms preprocess, 47.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 6 1s, 46.9ms
Speed: 1.7ms preprocess, 46.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 5 1s, 47.0ms
Speed: 1.9ms preprocess, 47.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 0s, 5 1s, 49.3ms
Speed: 2.0ms preprocess, 49.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 0s, 6 1s, 47.9ms
Speed: 1.8ms preprocess, 47.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 0s, 6 1s, 46.3ms
Speed: 2.2ms preprocess, 46.3ms inference, 0.5ms postprocess per image at 

left_shoulder   : x = 0.8872112035751343, y = 0.12732864916324615, z =   0.026311589404940605
right_shoulder  : x = 0.8307903409004211, y = 0.13525167107582092, z =    -0.0282799880951643
left_elbow      : x = 0.9086595773696899, y = 0.20158305764198303, z =    0.05048057809472084
right_elbow     : x = 0.8283863067626953, y = 0.20403462648391724, z =   -0.08911881595849991
left_pinky      : x = 0.9113696813583374, y = 0.21033860743045807, z =   0.049400389194488525
left_index      : x = 0.9085837602615356, y =   0.199906587600708, z =   0.046559061855077744
left_thumb      : x = 0.9064477682113647, y =  0.1989726573228836, z =    0.04951750859618187
left_wrist      : x =  0.910013735294342, y =  0.2054484337568283, z =    0.05035515874624252
right_pinky     : x = 0.8297922015190125, y = 0.18265698850154877, z =   -0.17372842133045197
right_index     : x =  0.828612208366394, y = 0.17418469488620758, z =    -0.1777363419532776
right_thumb     : x = 0.8305238485336304, y = 0.176037818193

0: 384x640 2 0s, 4 1s, 45.7ms
Speed: 1.9ms preprocess, 45.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 4 1s, 43.9ms
Speed: 1.8ms preprocess, 43.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)



...

0: 384x640 7 0s, 49.1ms
Speed: 1.9ms preprocess, 49.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 0s, 1 1, 47.6ms
Speed: 1.9ms preprocess, 47.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)



left_shoulder   : x = -0.0027830677572637796, y = 0.8004048466682434, z =  -0.10400804877281189
right_shoulder  : x =  0.0024537688586860895, y =  0.667957603931427, z =  -0.16365623474121094
left_elbow      : x =    -0.0555681474506855, y = 0.8057953715324402, z = 0.0034573108423501253
right_elbow     : x =   0.001331400009803474, y = 0.6071520447731018, z =  -0.19161777198314667
left_pinky      : x =  -0.006582809146493673, y = 0.7481143474578857, z =  -0.06783156841993332
left_index      : x =   0.004252280108630657, y = 0.7580367922782898, z =  -0.07610420882701874
left_thumb      : x =  0.0017759696347638965, y = 0.7608147859573364, z = -0.058660414069890976
left_wrist      : x =   -0.01341258641332388, y = 0.7635295391082764, z = -0.052727438509464264
right_pinky     : x =    0.07018720358610153, y = 0.5770177841186523, z =   -0.1661253720521927
right_index     : x =    0.07534913718700409, y = 0.5736017227172852, z =   -0.1790143847465515
right_thumb     : x =    0.0712161362171

0: 384x640 6 0s, 48.9ms
Speed: 1.8ms preprocess, 48.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 0s, 45.5ms
Speed: 1.9ms preprocess, 45.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



left_shoulder   : x =  0.03687359392642975, y =  0.5873756408691406, z =  -0.11451955884695053
right_shoulder  : x = 0.028599895536899567, y =  0.3354441523551941, z =  -0.03956488147377968
left_elbow      : x = -0.06557661294937134, y =  0.6618799567222595, z =  -0.03015952743589878
right_elbow     : x = -0.06954087316989899, y =  0.3052603006362915, z = 0.0032353783026337624
left_pinky      : x = -0.13912080228328705, y =  0.6442895531654358, z =  -0.06513090431690216
left_index      : x = -0.13416138291358948, y =  0.6292333006858826, z =  -0.07054810225963593
left_thumb      : x = -0.12663999199867249, y =  0.6225359439849854, z =  -0.06267962604761124
left_wrist      : x = -0.12058364599943161, y =  0.6396042108535767, z =  -0.05983405560255051
right_pinky     : x = -0.14101752638816833, y = 0.23809240758419037, z =  -0.07948718965053558
right_index     : x = -0.13895270228385925, y = 0.24740970134735107, z =   -0.1193142831325531
right_thumb     : x =  -0.1318705528974533, y = 0.

0: 384x640 9 0s, 47.3ms
Speed: 1.9ms preprocess, 47.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 0s, 48.8ms
Speed: 1.9ms preprocess, 48.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 0s, 47.7ms
Speed: 1.9ms preprocess, 47.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 0s, 47.2ms
Speed: 1.8ms preprocess, 47.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 0s, 46.9ms
Speed: 1.8ms preprocess, 46.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 0s, 47.9ms
Speed: 2.2ms preprocess, 47.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 0s, 51.0ms
Speed: 2.0ms preprocess, 51.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 0s, 48.8ms
Speed: 2.0ms preprocess, 48.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 0s, 51.9ms


In [6]:
# JUST THE HANDS
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
            
        # Save our image    
        cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [14]:
# THE HANDS AND THE POSE
# TODO: REMOVE THE mini hands from the pose
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose, mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        # Get image used for pose estimation
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            hand_landmarks = hand_results.multi_hand_landmarks

            # PRINTING THE POSE COORDINATES
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            print("SHOULDERS")
            print("Left_Shoulder --- x: " + str(left_shoulder.x) + "y: " + str(left_shoulder.y) +  "z: " + str(left_shoulder.z))
            print("Right_Shoulder --- x: " + str(right_shoulder.x) + "y: " + str(right_shoulder.y) + "z: " + str(right_shoulder.z))

            print("ELBOWS")
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            print("Left_elbow --- x: " + str(left_elbow.x) + "y: " + str(left_elbow.y) + "z: " + str(left_elbow.z))
            print("Right_elbow --- x: " + str(right_elbow.x) + "y: " + str(right_elbow.y) + "z: " + str(right_elbow.z))

            # LEFT HAND
            left_pinky = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value]
            left_index = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value]
            left_thumb = landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            print("LEFT HAND")
            print("Left_pinky --- x: " + str(left_pinky.x) + "y: " + str(left_pinky.y) + "z: " + str(left_pinky.z))
            print("left_index --- x: " + str(left_index.x) + "y: " + str(left_index.y) + "z: " + str(left_index.z))
            print("left_thumb --- x: " + str(left_thumb.x) + "y: " + str(left_thumb.y) + "z: " + str(left_thumb.z))
            print("Left_wrist --- x: " + str(left_wrist.x) + "y: " + str(left_wrist.y) + "z: " + str(left_wrist.z))

            # RIGHT HAND
            right_pinky = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value]
            right_index = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value]
            right_thumb = landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            print("RIGHT HAND")
            print("Right_pinky --- x: " + str(right_pinky.x) + "y: " + str(right_pinky.y) + "z: " + str(right_pinky.z))
            print("Right_index --- x: " + str(right_index.x) + "y: " + str(right_index.y) + "z: " + str(right_index.z))
            print("Right_thumb --- x: " + str(right_thumb.x) + "y: " + str(right_thumb.y) + "z: " + str(right_thumb.z))
            print("Right_wrist: x: " + str(right_wrist.x) + "y: " + str(right_wrist.y) + "z: " + str(right_wrist.z))

            # Lower body coordinates
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            print("LEFT KNEE --- x: " + str(left_knee.x) + " y: " + str(left_knee.y) + "z: " + str(left_knee.z))

            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            print("RIGHT KNEE --- x: " + str(right_knee.x) + " y: " + str(right_knee.y) + "z: " + str(right_knee.z))

            # LEFT FOOT
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]
            left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
            print("LEFT FOOT")
            print("Left_ankle --- x: " + str(left_ankle.x) + " y: " + str(left_ankle.y) + " z: " + str(left_ankle.z))
            print("Left_heel --- x: " + str(left_heel.x) + " y: " + str(left_heel.y) + " z: " + str(left_heel.z))
            print("Left_foot_index --- x: " + str(left_foot_index.x) + " y: " + str(left_foot_index.y) + " z: " + str(left_foot_index.z))

            # RIGHT FOOT
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value]
            right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
            print("RIGHT FOOT")
            print("Right_ankle --- x: " + str(right_ankle.x) + " y: " + str(right_ankle.y) + " z: " + str(right_ankle.z))
            print("Right_heel --- x: " + str(right_heel.x) + " y: " + str(right_heel.y) + " z: " + str(right_heel.z))
            print("Right_foot_index --- x: " + str(right_foot_index.x) + " y: " + str(right_foot_index.y) + " z: " + str(right_foot_index.z))

            frame_shape_0, frame_shape_1 = frame.shape[0], frame.shape[1]

            # DISPLAY AREA OF RIGHT FOOT ---------------
            right_foot_pts = np.array([
                [int(right_ankle.x * frame_shape_1), int(right_ankle.y * frame_shape_0)],
                [int(right_heel.x * frame_shape_1), int(right_heel.y * frame_shape_0)],
                [int(right_foot_index.x * frame_shape_1), int(right_foot_index.y * frame_shape_0)]
            ], np.int32).reshape((-1, 1, 2))

            # Fill the area of the right foot with a specific color
            cv2.fillPoly(image, [right_foot_pts], (245, 117, 66))

            # DISPLAY AREA OF LEFT FOOT ---------------
            left_foot_pts = np.array([
                [int(left_ankle.x * frame_shape_1), int(left_ankle.y * frame_shape_0)],
                [int(left_heel.x * frame_shape_1), int(left_heel.y * frame_shape_0)],
                [int(left_foot_index.x * frame_shape_1), int(left_foot_index.y * frame_shape_0)]
            ], np.int32).reshape((-1, 1, 2))

            # Fill the area of the left foot with a specific color
            cv2.fillPoly(image, [left_foot_pts], (245, 117, 66))
    
            # DISPLAY AREA OF RIGHT HAND ---------------
            right_hand_pts = np.array([
                [int(right_pinky.x * frame_shape_1), int(right_pinky.y * frame_shape_0), right_pinky.z],
                [int(right_index.x * frame_shape_1), int(right_index.y * frame_shape_0), right_index.z],
                [int(right_thumb.x * frame_shape_1), int(right_thumb.y * frame_shape_0), right_thumb.z],
                [int(right_wrist.x * frame_shape_1), int(right_wrist.y * frame_shape_0), right_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            right_center_3d = np.mean(right_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            right_distances = [np.linalg.norm([p[0] - right_center_3d[0], p[1] - right_center_3d[1], p[2] - right_center_3d[2]]) for p in right_hand_pts]
            radius_3d = int(sum(right_distances ) / len(right_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(right_center_3d[0]), int(right_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # DISPLAY AREA OF LEFT HAND ---------------
            left_hand_pts = np.array([
                [int(left_pinky.x * frame_shape_1), int(left_pinky.y * frame_shape_0), left_pinky.z],
                [int(left_index.x * frame_shape_1), int(left_index.y * frame_shape_0), left_index.z],
                [int(left_thumb.x * frame_shape_1), int(left_thumb.y * frame_shape_0), left_thumb.z],
                [int(left_wrist.x * frame_shape_1), int(left_wrist.y * frame_shape_0), left_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            left_center_3d = np.mean(left_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            left_distances = [np.linalg.norm([p[0] - left_center_3d[0], p[1] - left_center_3d[1], p[2] - left_center_3d[2]]) for p in left_hand_pts]
            radius_3d = int(sum(left_distances ) / len(left_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(left_center_3d[0]), int(left_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            # cv2.putText(image, str(angle),
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)

        except:
            pass

        # print(results.pose_landmarks)
        # Render detections

        cv2.imshow('Pose Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

SHOULDERS
Left_Shoulder --- x: 0.8209762573242188y: 0.8445996046066284z: -0.25761088728904724
Right_Shoulder --- x: 0.43199586868286133y: 0.7976332902908325z: -0.2880916893482208
ELBOWS
Left_elbow --- x: 0.9200819134712219y: 1.2825663089752197z: -0.2733474373817444
Right_elbow --- x: 0.3268992602825165y: 1.2658450603485107z: -0.3183847963809967
LEFT HAND
Left_pinky --- x: 0.9252463579177856y: 1.7560194730758667z: -0.865749716758728
left_index --- x: 0.8846017718315125y: 1.7459685802459717z: -0.942737877368927
left_thumb --- x: 0.8677908182144165y: 1.6975243091583252z: -0.787994921207428
Left_wrist --- x: 0.9064286351203918y: 1.6268954277038574z: -0.7346315979957581
RIGHT HAND
Right_pinky --- x: 0.2874073088169098y: 1.750937581062317z: -1.1176674365997314
Right_index --- x: 0.3335143029689789y: 1.7383403778076172z: -1.211480736732483
Right_thumb --- x: 0.3512462079524994y: 1.6903659105300903z: -1.0162590742111206
Right_wrist: x: 0.31628915667533875y: 1.6191951036453247z: -0.957507908344